<h1>Segmenting and Clustering Neighborhoods in Toronto: Activity #1</h1>
<h4>Getting Toronto neighborhoods data into a DataFrame</h4>
Get data of Toronto neighborhoods then display some rows along with shape head, finally, transform the data into a pandas dataframe.

In [1]:
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
import folium
import requests
import pandas as pd
import numpy as np

In [2]:
# Get data from the Wiki page
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text

# convert it into a table with beautifulsoup
canada_data = BeautifulSoup(source, 'lxml')

# loop through the data to populate a "table of contents"
table_contents = []
table = canada_data.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text == 'Not assigned':  # this ignore the cells with a borough that is Not assigned.
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /', ',')).replace(')', ' ')).strip(' ')
        table_contents.append(cell)

# Create a dataframe of toronto neighborhoods with the table of contents
torontoNHs = pd.DataFrame(table_contents)
# Replacing some str for their correct spelling
torontoNHs['Borough'] = torontoNHs['Borough'].replace({
    'Downtown TorontoStn A PO Boxes25 The Esplanade': 'Downtown Toronto Stn A',
    'East TorontoBusiness reply mail Processing Centre969 Eastern': 'East Toronto Business',
    'EtobicokeNorthwest': 'Etobicoke Northwest',
    'East YorkEast Toronto': 'East York/East Toronto',
    'MississaugaCanada Post Gateway Processing Centre': 'Mississauga'})
torontoNHs.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [3]:
torontoNHs.shape

(103, 3)

the dataframe has 103 rows

<h1>Segmenting and Clustering Neighborhoods in Toronto: Activity #2</h1>
<h4>Adding coordinates to the data frame</h4>
get the geographical coordinates of each neighborhood using the Geocoder package and add it to the dataframe

Using the csv file because of problems with geocoder

In [4]:
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv'
toronto_coords=pd.read_csv(url)
toronto_coords.shape

(103, 3)

As it has the same shape we can join the columns to our data matching the postal code 

In [5]:
torontoNHs = torontoNHs.join(toronto_coords.set_index('Postal Code'), on='PostalCode')
torontoNHs.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


<h1>Segmenting and Clustering Neighborhoods in Toronto: Activity #3</h1>
<h4>Exploring and cluster the neighborhoods in Toronto</h4>
 Only with boroughs that contain the word Toronto and replicating the same analysis we did to the New York City data. 

Getting the coordinates of Toronto to create the maps

In [6]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Creating the map

In [7]:
# create map of Toronto using latitude and longitude values
map_torontoNHs = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(torontoNHs['Latitude'], torontoNHs['Longitude'], torontoNHs['Borough'], torontoNHs['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        ).add_to(map_torontoNHs)
map_torontoNHs

In [8]:
# Getting a new data frame that only contais the entries that contain the word toronto in the borough
t = torontoNHs[torontoNHs['Borough'].str.contains("Toronto")]
t.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [9]:
t.shape

(39, 5)

Seeing how many unique boroughs are there and how many neighborhoods are

In [10]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(t['Borough'].unique()),
        t.shape[0]
    )
)

The dataframe has 7 boroughs and 39 neighborhoods.


Now a map that contains only the neighborhoods with the word Toronto in its borough

In [11]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(t['Latitude'], t['Longitude'], t['Borough'], t['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        ).add_to(map_Toronto)
map_Toronto

Define Foursquare Credentials and Version


In [12]:
CLIENT_ID = 'UMVNX4Y1RPNBYYB5KLXP22ZNPZD2RSY5YMC544FDFLIRU2ZR' # your Foursquare ID
CLIENT_SECRET = 'JFGNR2IPS2ROE2MGFJZOHLJWSCDBWSSFJW4LIEH21J3IKN04' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UMVNX4Y1RPNBYYB5KLXP22ZNPZD2RSY5YMC544FDFLIRU2ZR
CLIENT_SECRET:JFGNR2IPS2ROE2MGFJZOHLJWSCDBWSSFJW4LIEH21J3IKN04


Create the GET request URL (foursquare)

In [13]:
t.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [14]:
t.loc[2, 'Neighborhood']

'Regent Park, Harbourfront'

Explore the data, and get the venues in 500 meters range from our first entry

In [15]:
neighborhood_latitude = t.loc[2, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = t.loc[2, 'Longitude'] # neighborhood longitude value
neighborhood_name = t.loc[2, 'Neighborhood'] # neighborhood name
print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


Create the GET request URL

In [16]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=UMVNX4Y1RPNBYYB5KLXP22ZNPZD2RSY5YMC544FDFLIRU2ZR&client_secret=JFGNR2IPS2ROE2MGFJZOHLJWSCDBWSSFJW4LIEH21J3IKN04&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=100'

In [17]:
results = requests.get(url).json()

In [18]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [19]:
venues = results['response']['groups'][0]['items']

nearby_venues = pd.json_normalize(venues)  # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Tandem Coffee,Coffee Shop,43.653559,-79.361809
1,Roselle Desserts,Bakery,43.653447,-79.362017
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Impact Kitchen,Restaurant,43.656369,-79.356980


Generalize to obtain the venues from all neighborhoods in t

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)

        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']

        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']

    return(nearby_venues)

In [21]:
toronto_venues = getNearbyVenues(names=t['Neighborhood'], latitudes=t['Latitude'], longitudes = t['Longitude'])

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Enclave of M5E
St. James Town, Cabbagetown
First Canadi

print venues by Neighborhood



In [22]:
toronto_venues.groupby('Neighbourhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,46,46,46,46,46,46
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,62,62,62,62,62,62
Christie,15,15,15,15,15,15
Church and Wellesley,69,69,69,69,69,69
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,25,25,25,25,25,25
Davisville North,9,9,9,9,9,9


How many categories can we find?

In [23]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Get the top 10 for each neighbourhood

In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)

    return row_categories_sorted.index.values[0:num_top_venues]

In [25]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Sandwich Place,Seafood Restaurant,Farmers Market,Vegetarian / Vegan Restaurant,Beer Bar,Bistro,Spa
1,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Café,Sandwich Place,Breakfast Spot,Intersection,Convenience Store,Climbing Gym,Restaurant,Nightclub,Stadium
2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Terminal,Airport Lounge,Boat or Ferry,Rental Car Location,Coffee Shop,Sculpture Garden,Bar,Airport Gate,Airport Food Court
3,Central Bay Street,Coffee Shop,Sandwich Place,Sushi Restaurant,Café,Japanese Restaurant,Italian Restaurant,Restaurant,Burger Joint,Salad Place,Bank
4,Christie,Grocery Store,Café,Park,Athletics & Sports,Italian Restaurant,Nightclub,Restaurant,Baby Store,Coffee Shop,American Restaurant


Cluster neighborhoods

In [26]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 3, 0], dtype=int32)

Merge the dataframe with the top 10 and the cluster for each neighborhood

In [27]:
# add clustering labels
neighborhoods_venues_sorted.insert(2, 'Cluster Labels', kmeans.labels_)

toronto_merged = t

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,Cluster Labels,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,Coffee Shop,0,Park,Pub,Bakery,Café,Restaurant,Sandwich Place,Spa,Beer Store,Event Space
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,Coffee Shop,0,Sandwich Place,Clothing Store,Café,Pizza Place,Japanese Restaurant,Cosmetics Shop,Hotel,Bank,Thai Restaurant
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,Coffee Shop,0,Café,Italian Restaurant,Cocktail Bar,Gastropub,Restaurant,Clothing Store,Cosmetics Shop,Farmers Market,Seafood Restaurant
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,Health Food Store,4,Pub,Neighborhood,Yoga Studio,Deli / Bodega,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,Coffee Shop,0,Cocktail Bar,Bakery,Sandwich Place,Seafood Restaurant,Farmers Market,Vegetarian / Vegan Restaurant,Beer Bar,Bistro,Spa


In [28]:
import matplotlib.cm as cm
import matplotlib.colors as colors

toronto_merged[toronto_merged['Cluster Labels'].isnull()]

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

toronto_merged_nonan = toronto_merged.dropna(subset=['Cluster Labels'])

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged_nonan['Latitude'], toronto_merged_nonan['Longitude'], toronto_merged_nonan['Neighborhood'], toronto_merged_nonan['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters

Examine clusters

Cluster 1

In [29]:
toronto_merged_nonan.loc[toronto_merged_nonan['Cluster Labels'] == 0, toronto_merged_nonan.columns[[1] + list(range(5, toronto_merged_nonan.shape[1]))]]

,Borough,1st Most Common Venue,Cluster Labels,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,Coffee Shop,0,Park,Pub,Bakery,Café,Restaurant,Sandwich Place,Spa,Beer Store,Event Space
9,Downtown Toronto,Coffee Shop,0,Sandwich Place,Clothing Store,Café,Pizza Place,Japanese Restaurant,Cosmetics Shop,Hotel,Bank,Thai Restaurant
15,Downtown Toronto,Coffee Shop,0,Café,Italian Restaurant,Cocktail Bar,Gastropub,Restaurant,Clothing Store,Cosmetics Shop,Farmers Market,Seafood Restaurant
20,Downtown Toronto,Coffee Shop,0,Cocktail Bar,Bakery,Sandwich Place,Seafood Restaurant,Farmers Market,Vegetarian / Vegan Restaurant,Beer Bar,Bistro,Spa
24,Downtown Toronto,Coffee Shop,0,Sandwich Place,Sushi Restaurant,Café,Japanese Restaurant,Italian Restaurant,Restaurant,Burger Joint,Salad Place,Bank
25,Downtown Toronto,Grocery Store,0,Café,Park,Athletics & Sports,Italian Restaurant,Nightclub,Restaurant,Baby Store,Coffee Shop,American Restaurant
30,Downtown Toronto,Coffee Shop,0,Café,Sandwich Place,Clothing Store,Gym,Sushi Restaurant,Restaurant,Bank,Pizza Place,Burrito Place
31,West Toronto,Grocery Store,0,Middle Eastern Restaurant,Brewery,Bar,Bank,Bakery,Supermarket,Café,Liquor Store,Wine Shop
36,Downtown Toronto,Coffee Shop,0,Café,Hotel,Pizza Place,Scenic Lookout,Gym,Aquarium,Steakhouse,Italian Restaurant,Sandwich Place
37,West Toronto,Bar,0,Vietnamese Restaurant,Coffee Shop,New American Restaurant,Vegetarian / Vegan Restaurant,Asian Restaurant,Café,Diner,Japanese Restaurant,Boutique


Cluster 2

In [30]:
toronto_merged_nonan.loc[toronto_merged_nonan['Cluster Labels'] == 1, toronto_merged_nonan.columns[[1] + list(range(5, toronto_merged_nonan.shape[1]))]]

,Borough,1st Most Common Venue,Cluster Labels,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,Central Toronto,Fast Food Restaurant,1,Garden,Yoga Studio,Department Store,Escape Room,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run


Cluster 3

In [31]:
toronto_merged_nonan.loc[toronto_merged_nonan['Cluster Labels'] == 2, toronto_merged_nonan.columns[[1] + list(range(5, toronto_merged_nonan.shape[1]))]]

,Borough,1st Most Common Venue,Cluster Labels,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,East York/East Toronto,Intersection,2,Park,Convenience Store,Deli / Bodega,Escape Room,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run


Cluster 4

In [32]:
toronto_merged_nonan.loc[toronto_merged_nonan['Cluster Labels'] == 3, toronto_merged_nonan.columns[[1] + list(range(5, toronto_merged_nonan.shape[1]))]]

,Borough,1st Most Common Venue,Cluster Labels,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
61,Central Toronto,Business Service,3,Bus Line,Lawyer,Swim School,Park,Comic Shop,Concert Hall,Electronics Store,Eastern European Restaurant,College Rec Center
67,Central Toronto,Gym / Fitness Center,3,Hotel,Park,Breakfast Spot,Sandwich Place,Food & Drink Shop,Department Store,Dance Studio,Playground,Concert Hall
68,Central Toronto,Bus Line,3,Park,Trail,Jewelry Store,Sushi Restaurant,Yoga Studio,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run
91,Downtown Toronto,Park,3,Playground,Trail,Cupcake Shop,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


Cluster 5

In [33]:
toronto_merged_nonan.loc[toronto_merged_nonan['Cluster Labels'] == 4, toronto_merged_nonan.columns[[1] + list(range(5, toronto_merged_nonan.shape[1]))]]

,Borough,1st Most Common Venue,Cluster Labels,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,East Toronto,Health Food Store,4,Pub,Neighborhood,Yoga Studio,Deli / Bodega,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run
